# Consigna
 def UserForGenre( genero : str ): 
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
"Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]} 

In [20]:
import pandas as pd
import re

Para resolver este problema necesitare User_items y los Games

In [3]:
items_path = '../ETL/Data_Extracted/DataFrame_items.parquet'
ItemsDF = pd.read_parquet(items_path)

In [4]:
games_path = '../ETL/Data_Extracted/DataFrame_games.parquet'
GamesDF = pd.read_parquet(games_path)

In [5]:
# de aqui me interesan item_id user_id playtime_forever
GamesDF.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",lost summoner kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro


En base al DF anterior no puedo saber con exactitud cuantas horas jugo por año en el genero dado, lo que si puedo dar el total por genero

In [14]:
# de aqui me interesan item_id user_id playtime_forever
ItemsDF = ItemsDF[['item_id','playtime_forever','user_id']]
ItemsDF

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
1,20,0.0,76561197970982479
2,30,7.0,76561197970982479
3,40,0.0,76561197970982479
4,50,0.0,76561197970982479
...,...,...,...
5153204,346330,0.0,76561198329548331
5153205,373330,0.0,76561198329548331
5153206,388490,3.0,76561198329548331
5153207,521570,4.0,76561198329548331


In [8]:
# De aqui me interesan id del juego, los generos a los que pertenece, genre release_date
GamesDF = GamesDF[['genres','id','release_date']]
GamesDF

,genres,id,release_date
0,"[Action, Casual, Indie, Simulation, Strategy]",761140,2018-01-04
1,"[Free to Play, Indie, RPG, Strategy]",643980,2018-01-04
2,"[Casual, Free to Play, Indie, Simulation, Sports]",670290,2017-07-24
3,"[Action, Adventure, Casual]",767400,2017-12-07
4,"[Action, Adventure, Simulation]",772540,2018-01-04
...,...,...,...
27457,"[Action, Adventure, Casual, Indie]",745400,2018-01-04
27458,"[Casual, Indie, Simulation, Strategy]",773640,2018-01-04
27459,"[Casual, Indie, Strategy]",733530,2018-01-04
27460,"[Indie, Racing, Simulation]",610660,2018-01-04


In [9]:
#Cambio el nombre para poder unirlos por esa columna en comun
GamesDF = GamesDF.rename(columns={'id': 'item_id'})

In [15]:
#Agrego el precio a la informacion de mis usuarios
UserDataDF = pd.merge(GamesDF, ItemsDF, on='item_id' , how='left')

In [16]:
# Agrego las columnas que perdi en el merge
UserDataDF['playtime_forever'] = ItemsDF['playtime_forever']

In [17]:
UserDataDF['user_id'] = ItemsDF['user_id']

In [18]:
UserDataDF

,genres,item_id,release_date,playtime_forever,user_id
0,"[Action, Casual, Indie, Simulation, Strategy]",761140,2018-01-04,6.0,76561197970982479
1,"[Free to Play, Indie, RPG, Strategy]",643980,2018-01-04,0.0,76561197970982479
2,"[Casual, Free to Play, Indie, Simulation, Sports]",670290,2017-07-24,7.0,76561197970982479
3,"[Action, Adventure, Casual]",767400,2017-12-07,0.0,76561197970982479
4,"[Action, Adventure, Simulation]",772540,2018-01-04,0.0,76561197970982479
...,...,...,...,...,...
4116742,"[Action, Adventure, Casual, Indie]",745400,2018-01-04,6192.0,76561198025915551
4116743,"[Casual, Indie, Simulation, Strategy]",773640,2018-01-04,2953.0,76561198025915551
4116744,"[Casual, Indie, Strategy]",733530,2018-01-04,2093.0,76561198025915551
4116745,"[Indie, Racing, Simulation]",610660,2018-01-04,112.0,76561198025915551


Como solo me interesa el año, necesito sacarlo del release date

In [ ]:
# Definir la expresión regular para extraer el año
regex_pattern = r'(\d{4})'

# Aplicar la expresión regular a cada valor de la columna release_date y extraer el año
UserDataDF['release_year'] = UserDataDF['release_date'].apply(lambda x: re.search(regex_pattern, x).group(1) if re.search(regex_pattern, x) else None)

# Convertir la columna release_year a tipo numérico
UserDataDF['release_year'] = pd.to_numeric(UserDataDF['release_year'])


In [22]:
UserDataDF.drop(columns=['release_date'])

,genres,item_id,playtime_forever,user_id,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",761140,6.0,76561197970982479,2018.0
1,"[Free to Play, Indie, RPG, Strategy]",643980,0.0,76561197970982479,2018.0
2,"[Casual, Free to Play, Indie, Simulation, Sports]",670290,7.0,76561197970982479,2017.0
3,"[Action, Adventure, Casual]",767400,0.0,76561197970982479,2017.0
4,"[Action, Adventure, Simulation]",772540,0.0,76561197970982479,2018.0
...,...,...,...,...,...
4116742,"[Action, Adventure, Casual, Indie]",745400,6192.0,76561198025915551,2018.0
4116743,"[Casual, Indie, Simulation, Strategy]",773640,2953.0,76561198025915551,2018.0
4116744,"[Casual, Indie, Strategy]",733530,2093.0,76561198025915551,2018.0
4116745,"[Indie, Racing, Simulation]",610660,112.0,76561198025915551,2018.0


In [23]:
def UserForGenre(genero: str):
    # Filtramos por género
    data_genres = UserDataDF[UserDataDF['genres'].apply(lambda x: genero in x)]
     # Agrupamos por usuario y sumamos las horas jugadas
    data_playtime = data_genres.groupby('user_id')['playtime_forever'].sum().reset_index()
    # Obtenemos el usuario con más horas jugadas
    user = data_playtime.loc[data_playtime['playtime_forever'].idxmax()]['user_id']
    # Filtramos por usuario
        # Filtramos por usuario
    data_user = data_genres[data_genres['user_id'] == user]
    # Agrupamos por año y sumamos las horas jugadas
    data_year = data_user.groupby('release_year')['playtime_forever'].sum().reset_index()
    years = data_year.to_dict('records')
    # Obtenemos el año con más horas jugadas
    year = int(data_genres[data_genres['playtime_forever'] == data_genres['playtime_forever'].max()]['release_year'].values[0])
    
    return f"'Usuario con más horas jugadas para Género {genero}': {user}, 'Horas jugadas': {years}"

 

In [24]:
# prueba 
print(UserForGenre('Action'))

'Usuario con más horas jugadas para Género Action': Sp3ctre, 'Horas jugadas': [{'release_year': 2016.0, 'playtime_forever': 2822860.0}]


Exporto el DataFrame que necesito para cada consulta

In [ ]:
UserDataDF.to_parquet("data/UserForGenre.parquet")